> ### 在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成透视表

> #### pandas提供了一个灵活高效的gruopby功能，它使你能以一种自然的方式对数据集进行切片、切块、摘要等操作

> #### 使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象

> #### 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数

> #### 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等

> #### 计算透视表或交叉表

> #### 执行分位数分析以及其它统计分组分析

> **对时间序列数据的聚合（groupby的特殊用法之一）也称作重采样（resampling）**

In [1]:
import numpy as np
import pandas as pd
np.random.seed(2019)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

### 2.1 GroupBy机制

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],\
                   'key2' : ['one', 'two', 'one', 'two', 'one'],\
                   'data1' : np.random.randn(5),\
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.217679,0.685609,a,one
1,0.821455,0.573761,a,two
2,1.481278,0.287728,b,one
3,1.331864,-0.235634,b,two
4,-0.361865,0.953490,a,one


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.080637
b    1.406571
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.289772
      two     0.821455
b     one     1.481278
      two     1.331864
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.289772,0.821455
b,1.481278,1.331864


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.821455
            2006    1.481278
Ohio        2005    0.557093
            2006   -0.361865
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.080637,0.737620
b,1.406571,0.026047


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.289772  0.819550
     two   0.821455  0.573761
b    one   1.481278  0.287728
     two   1.331864 -0.235634

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

> ### 对分组进行迭代

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.217679  0.685609    a  one
1  0.821455  0.573761    a  two
4 -0.361865  0.953490    a  one
b
      data1     data2 key1 key2
2  1.481278  0.287728    b  one
3  1.331864 -0.235634    b  two


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.217679  0.685609    a  one
4 -0.361865  0.953490    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.821455  0.573761    a  two
('b', 'one')
      data1     data2 key1 key2
2  1.481278  0.287728    b  one
('b', 'two')
      data1     data2 key1 key2
3  1.331864 -0.235634    b  two


In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,1.481278,0.287728,b,one
3,1.331864,-0.235634,b,two


In [16]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.217679  0.685609
1  0.821455  0.573761
2  1.481278  0.287728
3  1.331864 -0.235634
4 -0.361865  0.953490
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


> ### 选取一列或列的子集

In [19]:
df.groupby('key1')['data1']

In [20]:
df.groupby('key1')[['data2']]

In [21]:
df['data1'].groupby(df['key1'])

In [22]:
df[['data2']].groupby(df['key1'])

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.819550
     two   0.573761
b    one   0.287728
     two  -0.235634

In [24]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [25]:
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     one     0.819550
      two     0.573761
b     one     0.287728
      two    -0.235634
Name: data2, dtype: float64

> ### 通过字典或Series进行分组

In [27]:
people = pd.DataFrame(np.random.randn(5, 5),\
                      columns=['a', 'b', 'c', 'd', 'e'],\
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-1.689625,-0.344943,0.016905,-0.514984,0.244509
Steve,-0.189313,2.672172,0.464802,0.845930,-0.503542
Wes,-0.963336,0.064969,-3.205040,1.054969,0.807277
Jim,0.474141,0.410928,0.486689,-0.535530,-0.838908
Travis,-0.812375,-0.450793,1.070801,0.217421,-1.175859


In [28]:
# 添加NAN值
people.iloc[2:3, [1, 2]] = np.nan 
people

,a,b,c,d,e
Joe,-1.689625,-0.344943,0.016905,-0.514984,0.244509
Steve,-0.189313,2.672172,0.464802,0.845930,-0.503542
Wes,-0.963336,NaN,NaN,1.054969,0.807277
Jim,0.474141,0.410928,0.486689,-0.535530,-0.838908
Travis,-0.812375,-0.450793,1.070801,0.217421,-1.175859


In [29]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',\
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [30]:
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [31]:
by_column = people.groupby(mapping, axis=1)
by_column

In [32]:
by_column.sum()

,blue,red
Joe,-0.498079,-1.790059
Steve,1.310733,1.979318
Wes,1.054969,-0.156059
Jim,-0.048840,0.046161
Travis,1.288223,-2.439026


In [33]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [35]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.178820,0.065986,0.503594,0.004456,0.212878
5,-0.189313,2.672172,0.464802,0.845930,-0.503542
6,-0.812375,-0.450793,1.070801,0.217421,-1.175859


In [36]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.689625 -0.344943  0.016905 -0.514984  0.244509
  two  0.474141  0.410928  0.486689 -0.535530 -0.838908
5 one -0.189313  2.672172  0.464802  0.845930 -0.503542
6 two -0.812375 -0.450793  1.070801  0.217421 -1.175859

> ### 根据索引级别分组

In [37]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],\
                                    [1, 3, 5, 1, 3]],\
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.926117 -0.993948  0.586806  1.063813  0.237741
1     -0.775057 -0.979104 -1.549363 -1.206828  0.445008
2     -0.173086  1.488947 -0.792520  1.838997 -0.439362
3      1.160003 -0.577093 -1.031044 -0.838885  0.316934

In [38]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2.2 数据聚合

In [40]:
df

,data1,data2,key1,key2
0,-0.217679,0.685609,a,one
1,0.821455,0.573761,a,two
2,1.481278,0.287728,b,one
3,1.331864,-0.235634,b,two
4,-0.361865,0.953490,a,one


In [41]:
grouped = df.groupby('key1')

In [42]:
grouped

In [43]:
grouped['data1'].quantile(0.9)

key1
a    0.613628
b    1.466336
Name: data1, dtype: float64

In [44]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.183321,0.379729
b,0.149414,0.523362


In [45]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.080637  0.645605 -0.361865 -0.289772 -0.217679  0.301888   
b      2.0  1.406571  0.105651  1.331864  1.369217  1.406571  1.443924   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.821455   3.0  0.737620  0.195134  0.573761  0.629685  0.685609   
b     1.481278   2.0  0.026047  0.370073 -0.235634 -0.104794  0.026047   

                          
           75%       max  
key1                      
a     0.819550  0.953490  
b     0.156887  0.287728

> ### 面向列的多函数应用

In [46]:
tips = pd.read_csv('Dataset/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [47]:
grouped = tips.groupby(['day', 'smoker'])

In [48]:
grouped

In [49]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [50]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [51]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [52]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [53]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [54]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [56]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [57]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],\
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

> ### 以“没有行索引”的形式返回聚合数据

In [58]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 2.3 apply：一般性的“拆分－应用－合并

In [59]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [60]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [61]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [62]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [63]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [ ]:
# f = lambda x: x.describe()
# grouped.apply(f)

> ## 禁止分组键

In [64]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


> ### 分位数和桶分析

In [65]:
frame = pd.DataFrame({'data1': np.random.randn(1000),\
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-0.0992, 1.414]
1       (1.414, 2.928]
2    (-1.613, -0.0992]
3    (-1.613, -0.0992]
4    (-1.613, -0.0992]
5       (1.414, 2.928]
6     (-3.133, -1.613]
7     (-0.0992, 1.414]
8     (-0.0992, 1.414]
9    (-1.613, -0.0992]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.133, -1.613] < (-1.613, -0.0992] < (-0.0992, 1.414] < (1.414, 2.928]]

In [66]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),\
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)

In [67]:
grouped

In [68]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.133, -1.613]",57.0,2.194161,-0.251959,-2.009601
"(-1.613, -0.0992]",402.0,3.031873,-0.018012,-3.940572
"(-0.0992, 1.414]",453.0,2.737978,-0.092133,-4.049539
"(1.414, 2.928]",88.0,2.143635,0.043378,-1.940773


In [69]:
# 获取分位数的编号
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.267874,-0.161253,-2.009601
1,100.0,2.681322,0.049217,-2.408535
2,100.0,2.603859,-0.035274,-2.384560
3,100.0,2.223264,-0.101402,-3.940572
4,100.0,3.031873,-0.092205,-2.049838
5,100.0,1.770582,-0.027131,-2.164075
6,100.0,2.496671,-0.009204,-2.431747
7,100.0,2.690479,-0.141737,-2.656585
8,100.0,2.737978,-0.040419,-2.654304


> ### 示例：用特定于分组的值填充缺失值

In [70]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.906656
2         NaN
3    1.176448
4         NaN
5   -0.828825
dtype: float64

In [71]:
s.fillna(s.mean())

0   -0.186344
1   -0.906656
2   -0.186344
3    1.176448
4   -0.186344
5   -0.828825
dtype: float64

In [72]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',\
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4

In [73]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.743621
New York     -0.074997
Vermont       0.436613
Florida      -0.959268
Oregon       -0.714441
Nevada        0.590417
California   -0.558222
Idaho        -0.256416
dtype: float64

In [74]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.743621
New York     -0.074997
Vermont            NaN
Florida      -0.959268
Oregon       -0.714441
Nevada             NaN
California   -0.558222
Idaho              NaN
dtype: float64

In [75]:
data.groupby(group_key).mean()

East   -0.592629
West   -0.636332
dtype: float64

In [76]:
fill_mean = lambda g: g.fillna(g.mean())

In [77]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.743621
New York     -0.074997
Vermont      -0.592629
Florida      -0.959268
Oregon       -0.714441
Nevada       -0.636332
California   -0.558222
Idaho        -0.636332
dtype: float64

In [78]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

In [79]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.743621
New York     -0.074997
Vermont       0.500000
Florida      -0.959268
Oregon       -0.714441
Nevada       -1.000000
California   -0.558222
Idaho        -1.000000
dtype: float64

> ### 示例：随机采样和排列

In [80]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [81]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [82]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

8S    8
AC    1
2S    2
5H    5
4C    4
dtype: int64

In [83]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  10C    10
   6C      6
D  6D      6
   2D      2
H  4H      4
   9H      9
S  QS     10
   KS     10
dtype: int64

In [84]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C     9
QC    10
7D     7
QD    10
8H     8
2H     2
3S     3
2S     2
dtype: int64

> ### 示例：分组加权平均数和相关系数

In [85]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',\
                                'b', 'b', 'b', 'b'],\
                   'data': np.random.randn(8),\
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.781571,0.328726
1,a,0.468435,0.240667
2,a,-0.599013,0.351743
3,a,1.395630,0.052365
4,b,0.799124,0.764315
5,b,-0.661124,0.499870
6,b,-0.400102,0.599319
7,b,-0.167935,0.513156


In [86]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.238360
b   -0.019212
dtype: float64

In [87]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [88]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [89]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [90]:
rets = close_px.pct_change().dropna()

In [91]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [92]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

> ### 示例：组级别的线性回归

In [93]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

D:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [94]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 2.4 透视表和交叉表

In [95]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [96]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],\
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [97]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],\
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [98]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',\
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [99]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],\
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

> ### 交叉表：crosstab

In [101]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

In [102]:
data = pd.read_table(StringIO(data), sep='\s+')

In [103]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [104]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [105]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## Conclusion

> **掌握 Pands 数据分组操作**

> **熟悉数据清理、建模和统计分析**

> **学会使用 GroupBy操作**